# Representing Networks

## Graph

Mathematically a network can be represented by an undirected graph. Every node represents a computer/router/bridge/switch/modem (machine), whereas the edges represent the connection between two machines. 

![Graph representation of a network](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/4x4_grid_spanning_tree.svg/220px-4x4_grid_spanning_tree.svg.png)

In reality connections do not have to be _bi-directional_, neither _symmetric_. Asymmetry in this case meaning that the latency/bandwidth/energy/etc.(_cost_) from node A to B is not equal to the cost from B to A. When discussing spanning trees we will take this into consideration when looking at the adjecency and/or incedence matrix of the graph (numeric representation of arbitrary graphs)


## Communication

Communication in your network is mathematically equivalent to _messages_ being passed along the graph. 

Lets consider an example graph:
![width = 0.35\linewidth](./example_necessity_for_trees.png)

Consider the problem to send a message from $A$ to $E$. Similar to a real network, the machines don't know the real lay-out of the network.
If we then want to send the message, the only thing we can do is send all the neighbours the message and tell them which node is the final destination.

Since the neighbors identity is also not known at this point, it becomes evident that the cycle $ABC$ will keep duplicating the package and will keep propagating the message ad infinitum.

If we write this graph in adjecency form s.t.:
messages incident on edge $E$ don't get passed along.
we obtain:

|   | A | B | C | D | E |
|---|---|---|---|---|---|
| A | 0 | 1 | 1 | 0 | 0 |
| B | 1 | 0 | 1 | 1 | 1 |
| C | 1 | 1 | 0 | 0 | 1 |
| D | 0 | 1 | 0 | 0 | 0 |
| E | 0 | 0 | 0 | 0 | 0 |

On this we can perform some computations

In [1]:
import numpy as np

A_mat = np.matrix([[0,1,1,0,0],[1, 0, 1,1,1],[1,1,0,0,1],[0,1,0,0,0],[0,0,0,0,0]])

m = np.zeros(5)
m[0]=1 #zero-indexed

print("Adjacency Matrix");print(A_mat);print("\n")
print("Message seen by the nodes at N=1");
print("A B C D E")
print(m);

#messages with naive/blind routing
N = 8
m_n = m*(A_mat**N)

print("messages after "+ str(N)+" SYNCHRONIZED transmissions")
print("A B C D E")
print(m_n)

Adjacency Matrix
[[0 1 1 0 0]
 [1 0 1 1 1]
 [1 1 0 0 1]
 [0 1 0 0 0]
 [0 0 0 0 0]]


Message seen by the nodes at N=1
A B C D E
[1. 0. 0. 0. 0.]
messages after 8 SYNCHRONIZED transmissions
A B C D E
[[137. 152. 136.  76. 137.]]


if we remove the cycles in the graph, this explosion of messages will vanish.
A way of adding that into this graph is to say that messages cannot be send between $B$ and $C$ and since $A$ is the origin, no messages are alowed to be incident on $A$

resulting in:

|   | A | B | C | D | E |
|---|---|---|---|---|---|
| A | 1 | 1 | 1 | 0 | 0 |
| B | 0 | 0 | 0 | 1 | 1 |
| C | 0 | 0 | 0 | 0 | 1 |
| D | 0 | 1 | 0 | 0 | 0 |
| E | 0 | 0 | 0 | 0 | 0 |

In [2]:
import numpy as np

A_mat = np.matrix([[1,1,1,0,0],[0, 0, 0,1,1],[0,0,0,0,1],[0,1,0,0,0],[0,0,0,0,0]])

m = np.zeros(5, np.int8)
m[0]=1 #zero-indexed

print("Adjacency Matrix");print(A_mat);print("\n")
print("Message seen by the nodes at N=1");
print("A B C D E")
print(m); print("\n")

#messages with naive/blind routing
N = 8
m_n = m*(A_mat**N)

print("messages after "+ str(N)+" SYNCHRONIZED transmissions")
print("A B C D E")
print(m_n)

Adjacency Matrix
[[1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 0 0 0]]


Message seen by the nodes at N=1
A B C D E
[1 0 0 0 0]


messages after 8 SYNCHRONIZED transmissions
A B C D E
[[1 4 1 4 5]]


We still observe that the package is duplicated, since $A$ sends the message to both $B$ and $C$.
following the path $ACE$ we don't see any duplication, but since $B$ sends the message to both $D$ and $E$ we get a duplication. Additionally, $B$ and $D$ keep sending the message back and forth, so it gets duplicated everytime the message reaches $B$.

If we further want to account for this, we can tell node $A$ not to send to node $B$ resulting in the adjacency matrix:

|   | A | B | C | D | E |
|---|---|---|---|---|---|
| A | 1 | 0 | 1 | 0 | 0 |
| B | 0 | 0 | 0 | 1 | 1 |
| C | 0 | 0 | 0 | 0 | 1 |
| D | 0 | 1 | 0 | 0 | 0 |
| E | 0 | 0 | 0 | 0 | 0 |

In [3]:
import numpy as np

A_mat = np.matrix([[1,0,1,0,0],[0, 0, 0,1,1],[0,0,0,0,1],[0,1,0,0,0],[0,0,0,0,0]])

m = np.zeros(5, np.int8)
m[0]=1 #zero-indexed

print("Adjacency Matrix");print(A_mat);print("\n")
print("Message seen by the nodes at N=1");
print("A B C D E")
print(m); print("\n")

#messages with naive/blind routing
N = 8
m_n = m*(A_mat**N)

print("messages after "+ str(N)+" SYNCHRONIZED transmissions")
print("A B C D E")
print(m_n)

Adjacency Matrix
[[1 0 1 0 0]
 [0 0 0 1 1]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 0 0 0]]


Message seen by the nodes at N=1
A B C D E
[1 0 0 0 0]


messages after 8 SYNCHRONIZED transmissions
A B C D E
[[1 0 1 0 1]]


Now we see desired behaviour. The message goes from $A$ to $C$ to $E$, and there is not a single node that sees the message more than ones

# Spanning Trees

Since we now have a desired behaviour, i.e.,

-No duplication

-Message reaches destination if both nodes are on the same graph

We can find a graph structure that fullfills these requirements. This structure is known as a _tree_.

A tree is a graph without cycles.
The tree we just used looks as follows:

![width = 0.35\linewidth](./example_first_spanning_trees.png)

In addition to this tree structure we will also consider that a node will never send the message back to who it is received from. Later we will see that devices do this through means of recognizing from which socket they received the message. 
(in the previous example this was implemented for messaged from node $A$ over the path $ACE$, but was left out in path $BD$ for demonstrative purposes)

Since the tree spans the entirety of the nodes of the real network's graph it is a special type of tree known as a _Spanning Tree_.

# Distance-vector routing protocol

The DVR-protocol used in some data networks is guaranteed to produce spanning trees. It is based on the Bellman-Ford algorithm. Modern networks use a different approach, but the general principle is very educational for understanding how to create a spanning tree, without knowing the absolute structure of the net. Consider the earlier example with costs:

![width = 0.35\linewidth](./example_tree_DVR.png)

The distance-vector routing protocol assumes that it can be known to every node what the _cost_ is to send a message to its neighbour. Additionally every node needs to have a unique name (in a real network this is the mac-address).

The DVR-protocol can run asynchronous (messages are sent without a synchronized timing), but for explanative reasons it will be described as if all nodes are synchronized.

step 1) initialize the cost to a neighbor, and all unknown nodes are set to $\infty$ (for practical reasons $\infty$ equals a big number, as to reduce computational complexity. we will use $500$)

step 2) inform the neighboring node of your known cost to your neighboring nodes

step 3) keep track of the cost to a node via a neighbour according to the formula: 
$\mathcal{L}_{new}(A \rightarrow B \leftarrow C) = min(\mathcal{L}_{old}(A \rightarrow B \rightarrow C), \mathcal{L}_{new}(B \rightarrow C)+\mathcal{L}_{old}(A \rightarrow B)$

step 4) repeat step 2 to 3 until convergence (or ad infinitum)

step 5) Send messages according to the shortest distance in your stored distance table.

![Example Of Distance Vector Routing](.\example_worked_DVR.png)